In [1]:
!pip install langchain
#!pip install chromadb
!pip install -U langchain-chroma
!pip install pypdf
!pip install pytest
!pip install ollama

In [28]:
import argparse
import os
import shutil
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
#from get_embedding_function import get_embedding_function
from langchain.vectorstores.chroma import Chroma

CHROMA_PATH = "chroma"

In [29]:
document_loader = PyPDFDirectoryLoader("documents")
documents = document_loader.load()
print(documents[1])

entry 27 in Xref table invalid but object found


page_content='Neuron, Vol. 36, 585–596, November 14, 2002, Copyright 2002 by Cell Press
The Unfolded Protein Response Modulates
Disease Severity in Pelizaeus-Merzbacher Disease
an X-linked recessive pediatric disorder characterized
by three common genetic forms of disease: coding re-gion or splice site mutations, duplications of the wild-Cherie M. Southwood,
1James Garbern,1,3
Wei Jiang,1and Alexander Gow1,2,3,4
1Center for Molecular Medicine and Genetics
type PLP1 gene, and null alleles. These mutations yield2Department of Pediatrics
a broad spectrum of disease phenotypes from severe,3Department of Neurology
connatal disease to mild forms characterized by pure Wayne State University School of Medicine
spastic paraparesis (reviewed in Garbern et al., 1999; Detroit, Michigan 48201
Southwood and Gow, 2001). Mutant alleles that model
all three of these genetic forms of PMD are available inmice, including (1) myelin synthesis-deficient (msd), an Summary
A242V missense mutation causing sev

In [30]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1400,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

chunks = text_splitter.split_documents(documents)
print(chunks[2])

page_content='tients (Kobayashi et al., 1994; Schneider et al., 1992); repression. Herein, we demonstrate UPR activation
(2)4e-Plp , comprising supernumerary copies of a 40 kb in the leukodystrophy Pelizaeus-Merzbacher disease
wild-type Plp1 transgene causing overexpression (Ka- (PMD) as well as in three mouse models of this disease
gawa et al., 1994); and (3) Plp1 null (Klugmann et al., and transfected fibroblasts expressing mutant protein.
1997; Rosenbluth et al., 1996; Stecca et al., 2000). Dis- The CHOP protein, widely known as a proapoptotic
ease in msd mice is apparent by 12 days postnatal (P12) transcription factor, modulates pathogenesis in the
as moderate tremors, which rapidly progress through mouse models of PMD; however, this protein exhibits
severe tremors to seizures causing death by 3–4 weeks. antiapoptotic activity. Together, these data show that
On the other hand, rshmice develop symptoms by P14– the UPR has the potential to modulate disease severity
16, exhibit modera

In [31]:
from langchain_community.embeddings import OllamaEmbeddings

ollama_emb = OllamaEmbeddings(model="llama3")

In [17]:
from langchain_chroma import Chroma

db = Chroma(persist_directory=CHROMA_PATH, embedding_function = ollama_emb)

last_page_id = None
current_chunk_index = 0

for chunk in chunks:
    source = chunk.metadata.get("source")
    page = chunk.metadata.get("page")
    current_page_id = f"{source}:{page}"

    # If the page ID is the same as the last one, increment the index.
    if current_page_id == last_page_id:
        current_chunk_index += 1
    else:
        current_chunk_index = 0

    # Calculate the chunk ID.
    chunk_id = f"{current_page_id}:{current_chunk_index}"
    last_page_id = current_page_id

    # Add it to the page meta-data.
    chunk.metadata["id"] = chunk_id

# Calculate Page IDs.
chunks_with_ids = chunks

# Add or Update the documents.
existing_items = db.get(include=[])  # IDs are always included by default
existing_ids = set(existing_items["ids"])
print(f"Number of existing documents in DB: {len(existing_ids)}")

# Only add documents that don't exist in the DB.
new_chunks = []
for chunk in chunks_with_ids:
    if chunk.metadata["id"] not in existing_ids:
        new_chunks.append(chunk)

if len(new_chunks):
    print(f"👉 Adding new documents: {len(new_chunks)}")
    new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
    db.add_documents(new_chunks, ids=new_chunk_ids)
else:
    print("✅ No new documents to add")

Number of existing documents in DB: 102
👉 Adding new documents: 3934


In [32]:
#CHatGPT
from langchain_chroma import Chroma

db = Chroma(persist_directory=CHROMA_PATH, embedding_function=ollama_emb)

last_page_id = None
current_chunk_index = 0

for chunk in chunks:
    source = chunk.metadata.get("source")
    page = chunk.metadata.get("page")
    current_page_id = f"{source}:{page}"

    # If the page ID is the same as the last one, increment the index.
    if current_page_id == last_page_id:
        current_chunk_index += 1
    else:
        current_chunk_index = 0

    # Calculate the chunk ID.
    chunk_id = f"{current_page_id}:{current_chunk_index}"
    last_page_id = current_page_id

    # Add it to the page meta-data.
    chunk.metadata["id"] = chunk_id

# Calculate Page IDs.
chunks_with_ids = chunks

# Add or Update the documents.
existing_items = db.get(include=[])  # IDs are always included by default
existing_ids = set(existing_items["ids"])
print(f"Number of existing documents in DB: {len(existing_ids)}")

# Only add documents that don't exist in the DB.
new_chunks = [chunk for chunk in chunks_with_ids if chunk.metadata["id"] not in existing_ids]

# Batch processing
BATCH_SIZE = 1000
try:
    if len(new_chunks):
        print(f"👉 Adding new documents: {len(new_chunks)}")
        for start in range(0, len(new_chunks), BATCH_SIZE):
            batch = new_chunks[start:start + BATCH_SIZE]
            batch_ids = [chunk.metadata["id"] for chunk in batch]
            db.add_documents(batch, ids=batch_ids)
        print("✅ New documents added successfully")
    else:
        print("✅ No new documents to add")
except Exception as e:
    print(f"Error occurred during ingestion: {e}")


Number of existing documents in DB: 1955
👉 Adding new documents: 73
✅ New documents added successfully


In [38]:
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama

PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

question="Give me the points in common of all diseases you know about."
#question="Do you know how chromosome affects Fabry's disease?"
#question="Is X chromosome and 22 the same?"
#expected_response="Holland or Chile."

# Prepare the DB.
#db = Chroma(persist_directory=CHROMA_PATH, embedding_function=ollama_emb)

# Search the DB.
results = db.similarity_search_with_score(question, k=8)

context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=question)
# print(prompt)

model = Ollama(model="llama3")
response_text = model.invoke(prompt)

sources = [doc.metadata.get("id", None) for doc, _score in results]
formatted_response = f"Response: {response_text}\nSources: {sources}"

print(formatted_response)

Response: Based on the provided context, I can identify some common points among the diseases mentioned:

1. Genetic origin: All the diseases mentioned have a genetic basis, with mutations or abnormalities in specific genes being the underlying cause.

2. Neurological involvement: Most of the diseases (Fabry disease, Pelizaeus-Merzbacher disease, and spastic paraplegia) affect the nervous system and are characterized by neurological symptoms such as muscle weakness, paralysis, and cognitive impairment.

3. Gene-related: The context mentions specific genes like PLP1, alpha-galactosidase A, and HLA types, which suggests that genetic factors play a crucial role in the development of these diseases.

4. Inheritance pattern: Some of the diseases (e.g., Fabry disease and Pelizaeus-Merzbacher disease) have an inherited component, with autosomal recessive or X-linked patterns of inheritance being reported.

5. Lack of effective treatments: The provided context does not suggest that there are e